In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import os
root = pathlib.Path(os.getcwd()).parent

exp_data_path = root.joinpath('Fig2\Exp_t=0.xlsx')
simulation_data_path = pathlib.Path("deviation_of_Jy.xlsx")
ideal_data_path = root.joinpath('Fig2\Ideal_t=0.xlsx')

exp_data = pd.read_excel(exp_data_path, sheet_name=None, index_col=0)
simulation_data = pd.read_excel(simulation_data_path, sheet_name=None, index_col=0)
ideal_data = pd.read_excel(ideal_data_path, sheet_name=None, index_col=0)

a4 = np.array([180, 297]) / 25.4
fontsize=8

mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amsmath}')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'

%matplotlib qt

In [2]:
x = np.linspace(0, 2**5 - 1, 2**5)
y = np.linspace(0, 2**5 - 1, 2**5)
xticks = np.linspace(0, 2**5 - 1, 3)
yticks = np.linspace(0, 2**5 - 1, 3)
xticklabels = [r'$-\pi$', r'$0$', r'$\pi$']
yticklabels = [r'$-\pi$', r'$0$', r'$\pi$']
# xticks = np.linspace(0, 2**5 - 1, 5)
# yticks = np.linspace(0, 2**5 - 1, 5)
# xticklabels = [r'$-\pi$', r'$-\pi/2$', r'$0$', r'$\pi/2$', r'$\pi$']
# yticklabels = [r'$-\pi$', r'$-\pi/2$', r'$0$', r'$\pi/2$', r'$\pi$']
xs, ys = np.meshgrid(x, y)
    
def set_ax(ax, xlabel=True, ylabel=True):
    ax.set_xlim(0, 2**5 - 1)
    ax.set_ylim(0, 2**5 - 1)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    if xlabel:
        ax.set_xticklabels(xticklabels, size=fontsize)
        ax.set_xlabel(r'$x$', size=fontsize)
    else:
        ax.set_xticklabels([])
    if ylabel:
        ax.set_yticklabels(yticklabels, size=fontsize)
        ax.set_ylabel(r'$y$', size=fontsize, labelpad=0)
    else:
        ax.set_yticklabels([])

fig = plt.figure(figsize=[a4[0], a4[1]/2])

density_clim=[0, 0.004]
momentum_clim=[0, 0.004]
density_cmap = 'Blues'
momentum_cmap = 'Greens'

streamplot_norm = mpl.colors.Normalize(vmin=0, vmax=0.002)
streamplot_cmap = "Greys"
ax_width = 0.17
ax_xs = np.array([0.1, 0.26, 0.5])+0.2
ax_ys = [0.7, 0.47, 0.31]

### simulation density step_0
density = np.array(simulation_data["density"])
ax = fig.add_axes([ax_xs[0], ax_ys[0], ax_width, ax_width])
im = ax.imshow(density, origin="lower", cmap=density_cmap, vmin=density_clim[0], vmax=density_clim[1])
set_ax(ax, xlabel=True, ylabel=True)


### simulation momentum step_0
momentum_x = np.array(simulation_data["momentum_x"])
momentum_y = np.array(simulation_data["momentum_y"])
ax = fig.add_axes([ax_xs[1], ax_ys[0], ax_width, ax_width])
im = ax.imshow((momentum_x**2+momentum_y**2)**0.5, origin="lower", cmap=momentum_cmap, vmin=momentum_clim[0], vmax=momentum_clim[1])
strm = ax.streamplot(xs,
                    ys,
                    momentum_x,
                    momentum_y,
                    density=0.7,
                    arrowsize=0.7,
                    color=np.sqrt(momentum_x**2+momentum_y**2),
                    norm=streamplot_norm,
                    cmap=streamplot_cmap,
                    linewidth=0.5)
set_ax(ax, xlabel=True, ylabel=False)

# jet density
cax = fig.add_axes([ax_xs[0]+0.04, ax_ys[0]+0.19, 0.09, 0.015])
cbar = plt.colorbar(mpl.cm.ScalarMappable(mpl.colors.Normalize(vmin=0, vmax=4), density_cmap), ax=ax, cax=cax, orientation='horizontal', ticks=[0, 4], location='top')
plt.xticks(fontsize=fontsize)
cax.tick_params('x', pad=0, length=3)
cax.text(2, 2.4, r"$\rho~(\text{\textperthousand})$", ha='center', va='center', fontsize=fontsize)
# jet momentum
cax = fig.add_axes([ax_xs[1]+0.04, ax_ys[0]+0.19, 0.09, 0.015])
cbar = plt.colorbar(mpl.cm.ScalarMappable(mpl.colors.Normalize(vmin=0, vmax=4), momentum_cmap), ax=ax, cax=cax, orientation='horizontal', ticks=[0, 4], location='top')
plt.xticks(fontsize=fontsize)
cax.tick_params('x', pad=0, length=3)
cax.text(2, 2.4, r"$|\boldsymbol{J}|~(\text{\textperthousand})$", ha='center', va='center', fontsize=fontsize)

ideal_color = 'k'
exp_color = "#007f00"
sim_color = "royalblue"
exp_marker = '^'
sim_marker = 'o'
exp_markerfacecolor = 'None'
sim_markerfacecolor = 'None'
exp_markersize=7
sim_markersize=5
exp_capsize=3
exp_alpha = 0.5
sim_alpha = 1

ax = fig.add_axes([ax_xs[2], ax_ys[0], ax_width*1.7, ax_width])
momentum_y_ideal = np.mean(np.array(ideal_data["momentum_y"]), axis=1)
momentum_y_sim = np.mean(np.array(simulation_data["momentum_y"]), axis=1)
momentum_y_exp = np.mean(np.array(exp_data["momentum_y"]), axis=1)
momentum_y_exp_std = np.std([np.mean(np.array(exp_data[f"momentum_y_rep{i}"]), axis=1) for i in range(1, 6)], axis=0, ddof=1)
y = np.mean(np.array(ideal_data["y"]), axis=1)
ax.plot(y, momentum_y_ideal*1000, ls='--', c=ideal_color, label='Ideal')
ax.errorbar(y, momentum_y_exp*1000, momentum_y_exp_std*1000, ls='-', marker=exp_marker, markersize=exp_markersize, markerfacecolor=exp_markerfacecolor, capsize=3, c=exp_color, alpha=exp_alpha, label='Exp.')
ax.plot(y, momentum_y_sim*1000, ls='-', c=sim_color, marker=sim_marker, markersize=sim_markersize, markerfacecolor=sim_markerfacecolor, alpha=sim_alpha, label='Sim.')
ax.ticklabel_format(style='scientific', scilimits=(-1, 0), axis='y', useMathText=True)
plt.ylim(-0.002*1000, 0.002*1000)
plt.xticks([-np.pi, 0, np.pi], [r'$-\pi$', r'$0$', r'$\pi$'], fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.xlim(-3.5, 3.5)
plt.xlabel(r'$y$', fontsize=fontsize)
plt.ylabel(r'$\langle J_y \rangle_x~(\text{\textperthousand})$', fontsize=fontsize, labelpad=-5)
plt.legend(fontsize=fontsize-1, ncols=3)


import re
from matplotlib import patches as pc
from matplotlib.axes import Axes
idx2qname = {
    0: 'q5_7',
    1: 'q3_9',
    2: 'q1_7',
    3: 'q1_9',
    4: 'q3_7',
    5: 'q7_7',
    6: 'q5_9',
    7: 'q7_11',
    8: 'q5_11',
    9: 'q7_9'
}
def position(name, reverse=True):
    idx1, idx2 = re.findall(r'\D*(\d*)_(\d*)', name)[0]
    if reverse:
        return int(idx2), int(idx1)
    else:
        return int(idx1), int(idx2)
location = np.array([position(idx2qname[idx]) for idx in range(10)])
location = (location-np.array([9, 4]))/8
color = {'normal':'#bdc1c6', 'error':'#ea4335'}
radius=0.09

def plot_layout(ax:Axes):
    for _idx, _loc in enumerate(location):
        fc = color['error']
        ax.add_patch(pc.Circle(_loc, radius, fc=fc, ec='None', lw=0.1))

ax = fig.add_axes([0.09, ax_ys[0], ax_width, ax_width])
plot_layout(ax)
width = 0.6
ax.set_xlim(-width, width)
ax.set_ylim(-width, width)
ax.set_aspect('equal')
ax.set_axis_off()


file_name = fr"SuppFigS7_mpl.pdf"
plt.savefig(file_name,
            dpi=600,
            facecolor='None',
            edgecolor='None',
            orientation='portrait',
            format="pdf",
            bbox_inches='tight')